# Beer Reviews 

This example analyzes beer reviews to find the most common words used in positive and negative reviews.
Original example can be found [here](https://medium.com/rapids-ai/real-data-has-strings-now-so-do-gpus-994497d55f8e).
The size of reviews_sample.csv is 23.1MB.
The full dataset is available on "s3://bodo-example-data/beer/reviews.csv" and its size is 2.2GB.

In [2]:
import pandas as pd
import time
import bodo

## Preprocessing
1. Create lists of stopwords and punctuation that will be removed.
2. Define regex that will be used to remove these punctuation and stopwords from the reviews.
3. Use the lower and strip functions to convert all letters to lowercase and remove excess whitespace. 
4. Remove stopwords and punctuation. 

In [3]:
with open("nltk-stopwords.txt", "r") as fh:
    STOPWORDS = list(map(str.strip, fh.readlines()))


PUNCT_LIST = ["\.", "\-", "\?", "\:", ":", "!", "&", "'", ","]
punc_regex = "|".join([f"({p})" for p in PUNCT_LIST])
stopword_regex = "|".join([f"\\b({s})\\b" for s in STOPWORDS])

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\-'
<>:5: SyntaxWarning: invalid escape sequence '\?'
<>:5: SyntaxWarning: invalid escape sequence '\:'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\-'
<>:5: SyntaxWarning: invalid escape sequence '\?'
<>:5: SyntaxWarning: invalid escape sequence '\:'
/var/folders/w_/z_0_fn150v36jdgzrrlcj8q00000gn/T/ipykernel_34309/1828026842.py:5: SyntaxWarning: invalid escape sequence '\.'
  PUNCT_LIST = ["\.", "\-", "\?", "\:", ":", "!", "&", "'", ","]
/var/folders/w_/z_0_fn150v36jdgzrrlcj8q00000gn/T/ipykernel_34309/1828026842.py:5: SyntaxWarning: invalid escape sequence '\-'
  PUNCT_LIST = ["\.", "\-", "\?", "\:", ":", "!", "&", "'", ","]
/var/folders/w_/z_0_fn150v36jdgzrrlcj8q00000gn/T/ipykernel_34309/1828026842.py:5: SyntaxWarning: invalid escape sequence '\?'
  PUNCT_LIST = ["\.", "\-", "\?", "\:", ":", "!", "&", "'", ","]
/var/folders/w_/z_0_fn150v3

In [4]:
@bodo.jit
def preprocess(reviews):
    # lowercase and strip
    reviews = reviews.str.lower()
    reviews = reviews.str.strip()

    # remove punctuation and stopwords
    reviews = reviews.str.replace(punc_regex, "", regex=True)
    reviews = reviews.str.replace(stopword_regex, "", regex=True)
    return reviews

## Find the Most Common Words

In [5]:
@bodo.jit
def find_top_words(review_filename):
    # Load in the data
    t_start = time.time()
    df = pd.read_csv(review_filename, parse_dates=[2])
    print("read time", time.time() - t_start)

    score = df.score
    reviews = df.text

    t1 = time.time()
    reviews = preprocess(reviews)
    print("preprocess time", time.time() - t1)

    t1 = time.time()
    # create low and high score series
    low_threshold = 1.5
    high_threshold = 4.95
    high_reviews = reviews[score > high_threshold]
    low_reviews = reviews[score <= low_threshold]
    high_reviews = high_reviews.dropna()
    low_reviews = low_reviews.dropna()

    high_colsplit = high_reviews.str.split()
    low_colsplit = low_reviews.str.split()
    print("high/low time", time.time() - t1)

    t1 = time.time()
    high_words = high_colsplit.explode()
    low_words = low_colsplit.explode()

    top_words = high_words.value_counts().head(25)
    low_words = low_words.value_counts().head(25)
    print("value_counts time", time.time() - t1)
    print("total time", time.time() - t_start)
    return top_words, low_words

    
top_words, low_words = find_top_words("s3://bodo-example-data/beer/reviews_sample.csv")

print(top_words)
print(low_words)

/Users/scottroutledge/Bodo/BodoSQL/bodosql/py4j_gateway.py:47: UserWarning: $JAVA_HOME is currently unset. This occurs when OpenJDK is not installed in your conda environment or when your environment has recently changed by not reactivates. BodoSQL will default to using you system's Java.It is recommended that you use OpenJDK v11 from Conda with BodoSQL. To do so, first run
    conda install openjdk=11 -c conda-forge
and then reactivate your environment via
    conda deactivate && conda activate /Users/scottroutledge/miniforge3
  warnings.warn(


read time 2.4556800000000294
preprocess time 8.18652700000007
high/low time 0.17107900000007703
value_counts time 0.02528899999992973
total time 10.83871899999997
text
beer         333
one          158
taste        140
head         119
like         117
best         102
dark          90
chocolate     90
great         86
perfect       80
good          79
sweet         77
smell         73
bottle        72
ive           70
flavor        68
glass         65
well          65
ever          65
aroma         64
nice          64
malt          63
beers         62
hops          62
bourbon       62
Name: count, dtype: int64
text
beer           239
like           109
taste          104
head            69
light           65
one             65
smell           57
bad             53
bottle          52
really          49
good            41
would           40
get             38
water           35
flavor          33
beers           32
much            32
smells          32
carbonation     32
glass          